# **Title Name : MNIST 데이터셋 모델링 연습**

<p style="font-weight:bolder; font-size : 21px">
    Step : Create file [step_1]
<p>
<p style="font-weight:bolder; font-size : 21px">
   RegDate : 2023.12.07

------------------------------------------------------------

용어정리
```
- 파라미터      : 세팅
- 함수,클래스   : 정의
- 클래스객체    : 생성
```

전체 프로세스
```
1.문제정의 >>> 2.EDA >>> 3.모델링 >>> 4.성능개선 >>> 5.모델링 >>> 6.제출 or 채점
```

진행순서(모델링 프로세스)



```
환경설정 >>> 전처리 >>> 모델생성 >>> optimizer,loss 생성 >>> 
[훈련 > 검증(생략가능) > 예측] >>> [모델저장(생략가능) > 모델불러오기](생략가능) >>> 
최종채점(생략가능) >>> 제출
```


```
1.  환경설정
2.  전처리
3.  모델생성
4.  optimizer,loss 생성
5.  훈련
6.  검증
7.  예측
8.  모델저장
9.  모델불러오기
10. 최종채점
11. 제출
```

# 1. 환경설정
------------------------------------------

In [1]:
#======================================================
# ▶ 모듈 불러오기
#======================================================

# 시스템
import os
import sys
import random
from time import time

# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 파이토치
from torch import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchsummary import summary

# CV용 토치비전
import torchvision
from torchvision import transforms
from torchvision.datasets import MNIST


# 사이킷런
import sklearn

# 검증용 Kfold
# import ~~~~~

# 유틸
import gc
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# 시각화테마
sns.set_theme(style='whitegrid')
plt.style.use('dark_background')

# 작업환경
IS_GOOGLE = True if 'google.colab'                in sys.modules  else False
IS_KAGGLE = True if 'KAGGLE_KERNEL_RUN_TYPE'      in os.environ   else False
IS_LOCAL  = True if not (IS_GOOGLE or IS_KAGGLE)                  else False

c:\anaconda3\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#======================================================
# ▶ 시드설정
#======================================================

SEED = 2023
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
torch.backends.cudnn.deterministic  =  True
torch.backends.cudnn.benchmark      =  False

In [3]:
#======================================================
# ▶ 데이터 경로 설정
#======================================================
if IS_GOOGLE :
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = '/content/drive/~~프로젝트패스~~설정~~/'
elif IS_KAGGLE :
    base_path = 'kaggle/input/프로젝트-패스/'
    
elif IS_LOCAL :
    # base_path = './data/'
    base_path = '../data/'

In [4]:
#======================================================
# ▶ 버전 체크
#======================================================

print(f'numpy_Ver   :  {np.__version__}버전')
print('-'*50)
print(f'pandas_Ver  :  {pd.__version__}버전')
print('-'*50)
print(f'seaborn_Ver :  {sns.__version__}버전')
print('-'*50)
print(f'torch_Ver   :  {torch.__version__}버전')
print('-'*50)
print(f'cpu_count   :  {os.cpu_count()}코어')

numpy_Ver   :  1.24.3버전
--------------------------------------------------
pandas_Ver  :  2.0.3버전
--------------------------------------------------
seaborn_Ver :  0.13.0버전
--------------------------------------------------
torch_Ver   :  2.1.1+cu118버전
--------------------------------------------------
cpu_count   :  12코어


# 2. 전처리
------------------------------------------

In [5]:
#======================================================
# ▶ 하이퍼파라미터 `초벌` 세팅
#======================================================
num_workers = int(os.cpu_count())-6 if os.cpu_count() else int(os.cpu_count())
batch_size = 16
learning_rate = 1e-2
epochs = 2

In [6]:
#======================================================
# ▶ 트랜스폼 객체생성
#======================================================
transform = transforms.ToTensor()

In [7]:
#======================================================
# ▶ 트테설정
#======================================================
train_dataset      =   MNIST(root=base_path, train=True,  transform=transform, download=True)
test_dataset       =   MNIST(root=base_path, train=False, transform=transform, download=True)
train_dataloader   =   DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True,  num_workers=num_workers)
test_dataloader    =   DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [8]:
#======================================================
# ▶ [데이터추출] 학습데이터 shape체크 및 추출
#======================================================
for X, y in train_dataloader:
    print(f'shape of tensor X [N,C,H,W] : {X.shape}')
    print(f'shape of tensor y           : {y.shape}')
    break

shape of tensor X [N,C,H,W] : torch.Size([16, 1, 28, 28])
shape of tensor y           : torch.Size([16])


# 3. 모델생성
------------------------------------------

In [9]:
#======================================================
# [스도코드작성] 모델 파라미터 플로우코드 
#======================================================

#~~~~~~~~~~~~~~~~~~~~~~~[NN]~~~~~~~~~~~~~~~~~~~~~~~~~~~
#   
# init    : (N,1,28,28) 
# flatten : (N,1*28*28) 
# Linear1 : (N,1*28*28) -> (N,1*28*28)
# Linear2 : (N,1*28*28) -> (N,1*28*28)
#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


#~~~~~~~~~~~~~~~~~~~~~~~[CNN]~~~~~~~~~~~~~~~~~~~~~~~~~~~
#
#----Conv1----
# (N,1,28,28)    ->  (N,6,28,28)    ->RELU->  (N,12,28,28)    -> (N,6,28,28) ->RELU-> 
#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [10]:
#======================================================
# ▶ [파라미터 세팅] 디바이스 세팅
#======================================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'
str_device = 'GPU' if torch.cuda.is_available() else 'CPU'
print(f'현재 사용 디바이스 : {str_device}')


현재 사용 디바이스 : GPU


In [11]:
#======================================================
# ▶ [파라미터세팅] 모델 파라미터 세팅
#======================================================

channel_size = int(X.shape[1])

# conv층
first_channel = channel_size
last_channel  = 64

# affine층
input_dim = 1*28*28
dense_dim = 512
num_classes = len(np.unique(next(iter(test_dataloader))[1]))

In [12]:
#======================================================
# ▶ [클래스정의] 모델 클래스 정의
#======================================================

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.relu    = nn.ReLU()
        self.linear1 = nn.Linear(input_dim,dense_dim)
        self.linear2 = nn.Linear(dense_dim,num_classes)
        self.linear_relu_stack = nn.Sequential(self.linear1,self.relu,self.linear2,self.relu)

    def forward(self, x):
        x = self.flatten(x)  
        logits = self.linear_relu_stack(x)
        return logits

In [13]:
#======================================================
# ▶ 모델 생성
#======================================================
model = NeuralNetwork().to(device)
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU()
  (linear1): Linear(in_features=784, out_features=512, bias=True)
  (linear2): Linear(in_features=512, out_features=9, bias=True)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=9, bias=True)
    (3): ReLU()
  )
)

In [14]:
#======================================================
# ▶ 피쳐 플로우 확인
#======================================================
summary(model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
            Linear-3                  [-1, 512]         401,920
              ReLU-4                  [-1, 512]               0
              ReLU-5                  [-1, 512]               0
            Linear-6                    [-1, 9]           4,617
            Linear-7                    [-1, 9]           4,617
              ReLU-8                    [-1, 9]               0
              ReLU-9                    [-1, 9]               0
Total params: 813,074
Trainable params: 813,074
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 3.10
Estimated Total Size (MB): 3.13
-------------------------------------------

# 3. optimizer 및 loss 설정   <!!!(훈련용 함수준비)>
----------------------------------------------------

In [15]:
#======================================================
# ▶ optimizer 객체생성
#======================================================
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [16]:
#======================================================
# ▶ loss function 객체생성
#======================================================
criterion = nn.BCELoss()

In [17]:
# #======================================================
# # ▶ 훈련용(train) 함수 정의
# #======================================================
# def train(dataloader,model,optimizer,criterion) :
#     '''
#     Info   : 훈련용 프로세스를 정의합니다(순전파->역전파->결과리턴)                                 \n
#     Params : 총 4개의 파라미터(Dataloader, Model, Optimizer, Loss Function)를 인자로 받습니다.      \n
#     Return : Average Loss(평균손실률)을 반환합니다.                                                 \n
#     '''
#     # 평가지표 초기화
#     total_loss = 0.0
    
#     # [학습용] 반복 추출
#     for batch , (X,y) in enumerate(dataloader) :
        
#         # 파라미터 
#         X = X.to(device)
#         y = y.to(device)
        
#         # 순전파
#         pred  = model(X)
#         loss  = criterion(pred,y)
#         total_loss += loss.item()
        
#         # 역전파
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
    
#     # 결과 출력
#     loss_avg = total_loss / len(dataloader)
#     return loss_avg

In [18]:
# #======================================================
# # ▶ 예측용(test) 함수 정의
# #======================================================
# def test(dataloader,model,criterion) :
#     '''
#     Info   : 훈련용 프로세스를 정의합니다(순전파->역전파->결과리턴)                                 \n
#     Params : 총 3개의 파라미터(Dataloader, Model, Optimizer, Loss Function)를 인자로 받습니다.      \n
#     Return : Average Loss(평균손실률)을 반환합니다.                                                 \n
#     '''
#     # 평가지표 초기화
#     total_loss = 0.0
    
#     # 추론모드(가중치 비활성)
#     with torch.no_grad() :

#         # [예측용] 반복 추출
#         for idx , (X,y) in enumerate(dataloader) :

#             # 파라미터
#             X = X.to(device)
#             y = y.to(device)

#             # Total loss Calculation
#             pred        = model(X)
#             loss        = criterion(pred,y)
#             total_loss += loss.item()


#         # 결과출력
#         loss_avg = total_loss / len(dataloader)
#         return loss_avg

# 4. 훈련
------------------------------------------------------

In [19]:
def train(dataloader, model, loss_fn, optimizer):
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred,y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            
            
def test(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, optimizer, criterion)
    test(test_dataloader, model, criterion)
print("Done!")
    

Epoch 1
-------------------------------


TypeError: 'Adam' object is not callable

In [19]:
#======================================================
# ▶ 훈련!
#======================================================

start = time()

# 미니배치 트레이닝
for epoch in tqdm(range(epochs)) :
    train(dataloader=train_dataloader,model=model,optimizer=optimizer,criterion=criterion)
    test(dataloader=test_dataloader,model=model,criterion=criterion)

end = time()


  0%|          | 0/2 [00:05<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
#======================================================
# ▶ 총 소요시간
#======================================================
elapsed_time =  end - start
hours   = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)
seconds = int(elapsed_time % 3600)
print(f' 총 소요시간 : {hours}시간 {minutes}분 {seconds}초')

NameError: name 'end' is not defined